In [1]:
import os
import sys
import numpy as np
root = os.path.dirname(os.path.abspath(os.curdir))
sys.path.append(root)

import torch
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import DataLoader
from torch.utils.data import Subset

from predify.utils.training import train_pcoders, eval_pcoders

from networks_2022 import BranchedNetwork
from data.CleanSoundsDataset import CleanSoundsDataset

# Parameters

In [2]:
DEVICE = 'cuda:0' if torch.cuda.is_available() else 'cpu'
print(f'Device: {DEVICE}')
BATCH_SIZE = 50
NUM_WORKERS = 2
PIN_MEMORY = True
NUM_EPOCHS = 50

lr = 1E-4
engram_dir = '/mnt/smb/locker/issa-locker/users/Erica/'
checkpoints_dir = f'{engram_dir}hcnn/checkpoints/'
tensorboard_dir = f'{engram_dir}hcnn/tensorboard/'
datafile = f'{engram_dir}seed_542_word_clean_random_order.hdf5'

Device: cuda:0


In [3]:
from pbranchednetwork_conv1 import PBranchedNetwork_Conv1SeparateHP
PNetClass = PBranchedNetwork_Conv1SeparateHP
pnet_name = 'conv1'

In [4]:
!nvidia-smi

Sat May 21 11:45:56 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.45.01    Driver Version: 455.45.01    CUDA Version: 11.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce RTX 208...  On   | 00000000:1F:00.0 Off |                  N/A |
| 27%   25C    P8    18W / 250W |      3MiB / 11019MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Load network and optimizer

In [ ]:
net = BranchedNetwork()
net.load_state_dict(torch.load(f'{engram_dir}networks_2022_weights.pt'))

/share/issa/users/es3773/temp-hallucnn/src/models/layers.py:78: UserWarning: Inconsistent tf pad calculation in ConvLayer.
  warnings.warn('Inconsistent tf pad calculation in ConvLayer.')
/share/issa/users/es3773/temp-hallucnn/src/models/layers.py:173: UserWarning: Inconsistent tf pad calculation: 0, 1
  warnings.warn(f'Inconsistent tf pad calculation: {pad_left}, {pad_right}')


<All keys matched successfully>

In [ ]:
pnet = PNetClass(net, build_graph=True)

In [ ]:
pnet.eval()

PBranchedNetwork_Conv1SeparateHP(
  (backbone): BranchedNetwork(
    (speech_branch): Sequential(
      (conv1): ConvLayer(
        (block): Sequential(
          (0): Conv2d(1, 96, kernel_size=(6, 14), stride=(3, 3), padding=(2, 6))
          (1): ReLU()
        )
      )
      (rnorm1): LRNorm(
        (block): LocalResponseNorm(5, alpha=0.005, beta=0.75, k=1.0)
      )
      (pool1): PoolLayer(
        (block): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
      )
      (conv2): ConvLayer(
        (block): Sequential(
          (0): Conv2d(96, 256, kernel_size=(5, 5), stride=(2, 2), padding=(1, 2))
          (1): ReLU()
        )
      )
      (rnorm2): LRNorm(
        (block): LocalResponseNorm(5, alpha=0.005, beta=0.75, k=1.0)
      )
      (pool2): PoolLayer(
        (block): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
      )
      (conv3): ConvLayer(
        (block): Sequential(
          (0): Conv2d(256, 512, kernel_siz

In [ ]:
for name, param in pnet.named_parameters():
    if 'genre_branch' in name:
        param.requires_grad = False

In [ ]:
pnet.to(DEVICE)
optimizer = torch.optim.Adam(
    [{'params':getattr(pnet,f"pcoder{x+1}").pmodule.parameters(), 'lr':lr} for x in range(pnet.number_of_pcoders)],
    weight_decay=5e-4)

# Set up dataset

In [ ]:
full_dataset = CleanSoundsDataset(datafile)
n_train = int(len(full_dataset)*0.9)
train_dataset = Subset(full_dataset, np.arange(n_train))
eval_dataset = Subset(full_dataset, np.arange(n_train, len(full_dataset)))
train_loader = DataLoader(
    train_dataset, batch_size=BATCH_SIZE, shuffle=True,
    num_workers=NUM_WORKERS, pin_memory=PIN_MEMORY
    )
eval_loader = DataLoader(
    eval_dataset, batch_size=BATCH_SIZE, shuffle=False,
    num_workers=NUM_WORKERS, pin_memory=PIN_MEMORY
    )

# Set up checkpoints and tensorboards

In [ ]:
checkpoint_path = os.path.join(checkpoints_dir, f"{pnet_name}")
if not os.path.exists(checkpoint_path):
    os.makedirs(checkpoint_path)
checkpoint_path = os.path.join(checkpoint_path, pnet_name + '-{epoch}-{type}.pth')

# summarywriter
from torch.utils.tensorboard import SummaryWriter
tensorboard_path = os.path.join(tensorboard_dir, f"{pnet_name}")
if not os.path.exists(tensorboard_path):
    os.makedirs(tensorboard_path)
sumwriter = SummaryWriter(tensorboard_path, filename_suffix=f'')

# Train

In [ ]:
loss_function = torch.nn.MSELoss()
for epoch in range(1, NUM_EPOCHS+1):
    train_pcoders(pnet, optimizer, loss_function, epoch, train_loader, DEVICE, sumwriter)
    eval_pcoders(pnet, loss_function, epoch, eval_loader, DEVICE, sumwriter)

    # save checkpoints every 5 epochs
    if epoch % 5 == 0:
        torch.save(pnet.state_dict(), checkpoint_path.format(epoch=epoch, type='regular'))

/home/es3773/.local/lib/python3.7/site-packages/torch/nn/functional.py:780: UserWarning: Note that order of the arguments: ceil_mode and return_indices will changeto match the args list in nn.MaxPool2d in a future release.
  warnings.warn("Note that order of the arguments: ceil_mode and return_indices will change"


Training Epoch: 1 [50/36450]	Loss: 168287.4219
Training Epoch: 1 [100/36450]	Loss: 159777.7188
Training Epoch: 1 [150/36450]	Loss: 179423.4688
Training Epoch: 1 [200/36450]	Loss: 164653.5469
Training Epoch: 1 [250/36450]	Loss: 173087.4375
Training Epoch: 1 [300/36450]	Loss: 148132.1406
Training Epoch: 1 [350/36450]	Loss: 163822.8125
Training Epoch: 1 [400/36450]	Loss: 148009.9062
Training Epoch: 1 [450/36450]	Loss: 151031.3281
Training Epoch: 1 [500/36450]	Loss: 160897.9844
Training Epoch: 1 [550/36450]	Loss: 147012.3125
Training Epoch: 1 [600/36450]	Loss: 137536.6250
Training Epoch: 1 [650/36450]	Loss: 141199.2344
Training Epoch: 1 [700/36450]	Loss: 135750.0000
Training Epoch: 1 [750/36450]	Loss: 138057.7812
Training Epoch: 1 [800/36450]	Loss: 131865.4531
Training Epoch: 1 [850/36450]	Loss: 125987.0156
Training Epoch: 1 [900/36450]	Loss: 132285.9531
Training Epoch: 1 [950/36450]	Loss: 120428.0156
Training Epoch: 1 [1000/36450]	Loss: 128298.9219
Training Epoch: 1 [1050/36450]	Loss: 131

Training Epoch: 1 [8650/36450]	Loss: 4123.2686
Training Epoch: 1 [8700/36450]	Loss: 4065.0554
Training Epoch: 1 [8750/36450]	Loss: 4046.7588
Training Epoch: 1 [8800/36450]	Loss: 4258.7891
Training Epoch: 1 [8850/36450]	Loss: 3939.5586
Training Epoch: 1 [8900/36450]	Loss: 4022.9951
Training Epoch: 1 [8950/36450]	Loss: 3689.8604
Training Epoch: 1 [9000/36450]	Loss: 3787.5610
Training Epoch: 1 [9050/36450]	Loss: 3716.4819
Training Epoch: 1 [9100/36450]	Loss: 3557.8987
Training Epoch: 1 [9150/36450]	Loss: 3510.5942
Training Epoch: 1 [9200/36450]	Loss: 3514.7483
Training Epoch: 1 [9250/36450]	Loss: 3357.9426
Training Epoch: 1 [9300/36450]	Loss: 3454.7314
Training Epoch: 1 [9350/36450]	Loss: 3335.9893
Training Epoch: 1 [9400/36450]	Loss: 3304.8359
Training Epoch: 1 [9450/36450]	Loss: 3211.7666
Training Epoch: 1 [9500/36450]	Loss: 3233.5747
Training Epoch: 1 [9550/36450]	Loss: 3247.4304
Training Epoch: 1 [9600/36450]	Loss: 3205.2983
Training Epoch: 1 [9650/36450]	Loss: 3104.6108
Training Epoc

Training Epoch: 1 [17250/36450]	Loss: 1337.2622
Training Epoch: 1 [17300/36450]	Loss: 1331.3682
Training Epoch: 1 [17350/36450]	Loss: 1311.2754
Training Epoch: 1 [17400/36450]	Loss: 1327.5210
Training Epoch: 1 [17450/36450]	Loss: 1333.0173
Training Epoch: 1 [17500/36450]	Loss: 1309.3784
Training Epoch: 1 [17550/36450]	Loss: 1259.2677
Training Epoch: 1 [17600/36450]	Loss: 1284.1460
Training Epoch: 1 [17650/36450]	Loss: 1346.9628
Training Epoch: 1 [17700/36450]	Loss: 1317.6759
Training Epoch: 1 [17750/36450]	Loss: 1220.9967
Training Epoch: 1 [17800/36450]	Loss: 1218.8387
Training Epoch: 1 [17850/36450]	Loss: 1263.6691
Training Epoch: 1 [17900/36450]	Loss: 1293.1213
Training Epoch: 1 [17950/36450]	Loss: 1296.8545
Training Epoch: 1 [18000/36450]	Loss: 1324.0638
Training Epoch: 1 [18050/36450]	Loss: 1226.8284
Training Epoch: 1 [18100/36450]	Loss: 1205.6410
Training Epoch: 1 [18150/36450]	Loss: 1187.1520
Training Epoch: 1 [18200/36450]	Loss: 1283.8835
Training Epoch: 1 [18250/36450]	Loss: 12

Training Epoch: 1 [25950/36450]	Loss: 926.3012
Training Epoch: 1 [26000/36450]	Loss: 982.1078
Training Epoch: 1 [26050/36450]	Loss: 927.6266
Training Epoch: 1 [26100/36450]	Loss: 924.4004
Training Epoch: 1 [26150/36450]	Loss: 836.8755
Training Epoch: 1 [26200/36450]	Loss: 851.0964
Training Epoch: 1 [26250/36450]	Loss: 881.3607
Training Epoch: 1 [26300/36450]	Loss: 951.1378
Training Epoch: 1 [26350/36450]	Loss: 872.4000
Training Epoch: 1 [26400/36450]	Loss: 886.5232
Training Epoch: 1 [26450/36450]	Loss: 831.1929
Training Epoch: 1 [26500/36450]	Loss: 912.8802
Training Epoch: 1 [26550/36450]	Loss: 847.7134
Training Epoch: 1 [26600/36450]	Loss: 886.5099
Training Epoch: 1 [26650/36450]	Loss: 887.0460
Training Epoch: 1 [26700/36450]	Loss: 998.7069
Training Epoch: 1 [26750/36450]	Loss: 879.7378
Training Epoch: 1 [26800/36450]	Loss: 889.5944
Training Epoch: 1 [26850/36450]	Loss: 959.5894
Training Epoch: 1 [26900/36450]	Loss: 917.0466
Training Epoch: 1 [26950/36450]	Loss: 867.4225
Training Epoc

...

Training Epoch: 8 [26750/36450]	Loss: 367.8198
Training Epoch: 8 [26800/36450]	Loss: 343.4922
Training Epoch: 8 [26850/36450]	Loss: 375.9127
Training Epoch: 8 [26900/36450]	Loss: 370.9811
Training Epoch: 8 [26950/36450]	Loss: 373.3600
Training Epoch: 8 [27000/36450]	Loss: 344.0076
Training Epoch: 8 [27050/36450]	Loss: 377.3595
Training Epoch: 8 [27100/36450]	Loss: 387.1175
Training Epoch: 8 [27150/36450]	Loss: 379.5851
Training Epoch: 8 [27200/36450]	Loss: 383.0186
Training Epoch: 8 [27250/36450]	Loss: 330.4539
Training Epoch: 8 [27300/36450]	Loss: 361.1277
Training Epoch: 8 [27350/36450]	Loss: 366.4926
Training Epoch: 8 [27400/36450]	Loss: 362.8041
Training Epoch: 8 [27450/36450]	Loss: 344.3589
Training Epoch: 8 [27500/36450]	Loss: 343.8816
Training Epoch: 8 [27550/36450]	Loss: 369.5201
Training Epoch: 8 [27600/36450]	Loss: 353.1656
Training Epoch: 8 [27650/36450]	Loss: 354.6327
Training Epoch: 8 [27700/36450]	Loss: 387.7417
Training Epoch: 8 [27750/36450]	Loss: 351.8202
Training Epoc

Training Epoch: 8 [35550/36450]	Loss: 380.8297
Training Epoch: 8 [35600/36450]	Loss: 361.0313
Training Epoch: 8 [35650/36450]	Loss: 347.9636
Training Epoch: 8 [35700/36450]	Loss: 352.1636
Training Epoch: 8 [35750/36450]	Loss: 378.7878
Training Epoch: 8 [35800/36450]	Loss: 354.2153
Training Epoch: 8 [35850/36450]	Loss: 369.1340
Training Epoch: 8 [35900/36450]	Loss: 337.2881
Training Epoch: 8 [35950/36450]	Loss: 352.1578
Training Epoch: 8 [36000/36450]	Loss: 356.8338
Training Epoch: 8 [36050/36450]	Loss: 354.7770
Training Epoch: 8 [36100/36450]	Loss: 323.3962
Training Epoch: 8 [36150/36450]	Loss: 335.8556
Training Epoch: 8 [36200/36450]	Loss: 355.9166
Training Epoch: 8 [36250/36450]	Loss: 356.6859
Training Epoch: 8 [36300/36450]	Loss: 373.8029
Training Epoch: 8 [36350/36450]	Loss: 331.7831
Training Epoch: 8 [36400/36450]	Loss: 369.3821
Training Epoch: 8 [36450/36450]	Loss: 352.7587
Training Epoch: 8 [4050/4050]	Loss: 163.6308
Training Epoch: 9 [50/36450]	Loss: 340.6310
Training Epoch: 9 

Training Epoch: 9 [8050/36450]	Loss: 353.5472
Training Epoch: 9 [8100/36450]	Loss: 355.1361
Training Epoch: 9 [8150/36450]	Loss: 367.5594
Training Epoch: 9 [8200/36450]	Loss: 376.8632
Training Epoch: 9 [8250/36450]	Loss: 335.9279
Training Epoch: 9 [8300/36450]	Loss: 334.5414
Training Epoch: 9 [8350/36450]	Loss: 387.2309
Training Epoch: 9 [8400/36450]	Loss: 368.5381
Training Epoch: 9 [8450/36450]	Loss: 376.4660
Training Epoch: 9 [8500/36450]	Loss: 370.5032
Training Epoch: 9 [8550/36450]	Loss: 374.9797
Training Epoch: 9 [8600/36450]	Loss: 341.0122
Training Epoch: 9 [8650/36450]	Loss: 368.9959
Training Epoch: 9 [8700/36450]	Loss: 344.7373
Training Epoch: 9 [8750/36450]	Loss: 381.0357
Training Epoch: 9 [8800/36450]	Loss: 350.7997
Training Epoch: 9 [8850/36450]	Loss: 358.6536
Training Epoch: 9 [8900/36450]	Loss: 377.0323
Training Epoch: 9 [8950/36450]	Loss: 398.5573
Training Epoch: 9 [9000/36450]	Loss: 360.1878
Training Epoch: 9 [9050/36450]	Loss: 358.0352
Training Epoch: 9 [9100/36450]	Los

Training Epoch: 9 [16850/36450]	Loss: 344.6807
Training Epoch: 9 [16900/36450]	Loss: 297.9201
Training Epoch: 9 [16950/36450]	Loss: 316.5976
Training Epoch: 9 [17000/36450]	Loss: 355.4341
Training Epoch: 9 [17050/36450]	Loss: 354.2213
Training Epoch: 9 [17100/36450]	Loss: 334.4082
Training Epoch: 9 [17150/36450]	Loss: 364.9858
Training Epoch: 9 [17200/36450]	Loss: 355.5107
Training Epoch: 9 [17250/36450]	Loss: 323.1617
Training Epoch: 9 [17300/36450]	Loss: 372.5023
Training Epoch: 9 [17350/36450]	Loss: 363.2270
Training Epoch: 9 [17400/36450]	Loss: 340.1024
Training Epoch: 9 [17450/36450]	Loss: 390.1930
Training Epoch: 9 [17500/36450]	Loss: 309.9747
Training Epoch: 9 [17550/36450]	Loss: 335.0842
Training Epoch: 9 [17600/36450]	Loss: 360.5982
Training Epoch: 9 [17650/36450]	Loss: 332.1317
Training Epoch: 9 [17700/36450]	Loss: 371.0581
Training Epoch: 9 [17750/36450]	Loss: 327.7363
Training Epoch: 9 [17800/36450]	Loss: 342.6143
Training Epoch: 9 [17850/36450]	Loss: 315.7322
Training Epoc

Training Epoch: 9 [25650/36450]	Loss: 342.8751
Training Epoch: 9 [25700/36450]	Loss: 370.5941
Training Epoch: 9 [25750/36450]	Loss: 331.4068
Training Epoch: 9 [25800/36450]	Loss: 317.6865
Training Epoch: 9 [25850/36450]	Loss: 363.2369
Training Epoch: 9 [25900/36450]	Loss: 353.1353
Training Epoch: 9 [25950/36450]	Loss: 347.5164
Training Epoch: 9 [26000/36450]	Loss: 341.0340
Training Epoch: 9 [26050/36450]	Loss: 326.7566
Training Epoch: 9 [26100/36450]	Loss: 326.4932
Training Epoch: 9 [26150/36450]	Loss: 318.2413
Training Epoch: 9 [26200/36450]	Loss: 338.1338
Training Epoch: 9 [26250/36450]	Loss: 338.6182
Training Epoch: 9 [26300/36450]	Loss: 348.5243
Training Epoch: 9 [26350/36450]	Loss: 357.3837
Training Epoch: 9 [26400/36450]	Loss: 373.0986
Training Epoch: 9 [26450/36450]	Loss: 328.8445
Training Epoch: 9 [26500/36450]	Loss: 340.0941
Training Epoch: 9 [26550/36450]	Loss: 365.7397
Training Epoch: 9 [26600/36450]	Loss: 319.4572
Training Epoch: 9 [26650/36450]	Loss: 328.8683
Training Epoc